<h1> SGD and GD Code </h1>

### (On MNIST) -- Includes comparison plots

### On bar above, navigate to Cell --> Run All. This will run all the cells.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker
import math
from math import *
import time
import random 
from numpy import linalg as LA
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix, precision_score, recall_score,f1_score
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
mnist = fetch_openml('mnist_784', version=1) #This is for fetching the MNIST data
mnist.keys()
U0, v0 = mnist["data"], mnist["target"]

C:\Users\RyanS\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
U = U0.astype(np.double) # changes the attribute values to float
v = v0.astype(np.uint8) # changes labels from string to integer
print(U.shape,"  ", v.shape)

(70000, 784)    (70000,)


In [4]:
# Change labels 
#Change the == value to an int inclusive 0 to 9 to binary classify that number
v_bin_5_lst = [2*int(v[i]==5)-1 for i in range(len(v))]   #{-1，1} for LogReg

# Data preparation: train and test data

In [5]:
# Create a single dataframe of all attributes and their labels
import pandas as pd
df_U = pd.DataFrame(data=U)
df_v = pd.DataFrame(data=np.asarray(v_bin_5_lst),  columns=['label'])
df_data_merged =pd.concat([df_U, df_v.reindex(df_U.index)], axis=1)
#df_data_merged.head(10)
#df_data_merged.info()

# Split the merged data randomly to train and test
def split_train_test(df_data_merged, train_set_size,test_set_size):
    np.random.seed(0) # use this line if you want your train and test data remain the same accross experiments
    shuffled_indices = np.random.permutation(len(df_data_merged))
    
    dic_train_set = {}
    dic_train_set_indices = shuffled_indices[:train_set_size]
    dic_train_set[0] = df_data_merged.iloc[dic_train_set_indices]
    
    dic_test_set= {}
    test_indices = shuffled_indices[-test_set_size:]
    dic_test_set[0] = df_data_merged.iloc[test_indices]
    
    return dic_train_set, dic_test_set

# Split the train or test data to U_train and v_train
def split_train_or_test_df_to_U_v(train_or_test_df):
    npar_data= (train_or_test_df).to_numpy()
    nparr_data_transp = npar_data.T
    U_output = nparr_data_transp[:-1,:]
    v_output =nparr_data_transp[-1:,:]
    return U_output,v_output

In [6]:
train_set_size,test_set_size = 1000,100  # you can adjust the train size here

dic_train_set, dic_test_set = split_train_test(df_data_merged, train_set_size,test_set_size)
#dic_train_set[0].head()

U_train,v_train = split_train_or_test_df_to_U_v(dic_train_set[0]) #Training Variables
n_train , N_train = np.shape(U_train)

U_test,v_test = split_train_or_test_df_to_U_v(dic_test_set[0]) #Testing variables
n_test , N_test = np.shape(U_test)

n = n_train # dimension / features
N = N_train # observations / samples

print(n,N)
print(U_train.shape,v_train.shape)

784 1000
(784, 1000) (1, 1000)


### Here U_train.T would be your $A$, v_train.T would be your $b$.
Do $GD$ for at least 2000 iterations, adjust the choice of stepsize (you can start with 0.001) until you get convergence.

# Gradient Descent Code

In [7]:
def gradient(x):
    #np.dot is used to perform matrix multiplication. DON'T use it on scalars
    result1 =  sum(localGradient(x, j) for j in range(N))#This is the gradient of an Ax - b function for GD
    return result1

In [8]:
def gradient_descent(x0, step_size, kmax, epoch):
    xnow = x0
    value = np.zeros(epoch+1)
    value[0] = logisticRegression(xnow)
    counter = 1 #Counter for the if statement present in the for loop
    for k in range(kmax):
        xnow = xnow - step_size * gradient(xnow)
        
        #This is for collecting data that will be used in the graph
        if ((k+1) % math.ceil(kmax / epoch)) == 0:
            value[counter] = logisticRegression(xnow) 
            counter +=1
    
    
    return xnow, value #xnow is final position, value is array of data points for graphs

# Stochastic Gradient Descent Code

In [9]:
#Function that finds the local gradient given a position x and randomized integer j. 
#For SGD
def localGradient(x, j):
    result = ((-U_train[:,[j]] * v_train[:,[j]]) / (1 + np.exp(-v_train[:,[j]] * np.dot(U_train.T[[j],:], x)))) / N
    return result

In [10]:
#Stochastic Gradient Descent function
def SGD(x0, step_size, kmax, epoch):
    xnow = x0
    value = np.zeros(epoch+1)
    value[0] = logisticRegression(xnow)
    counter = 1 #Counter for if statement in the for loop
    for k in range(kmax):
        xi = random.randint(0, N-1) #Randomizing a integer for SGD. Recall that N is total digits being tested
        xnow = xnow - step_size * localGradient(xnow, xi)
            
        #This is for collecting data that will be used in the graph
        if ((k+1) % math.ceil(kmax / epoch)) == 0:
            value[counter] = logisticRegression(xnow) 
            counter +=1
    
    
    return xnow, value

# Objective Function

In [11]:
def logisticRegression(x):
    return (sum(np.log(1 + np.exp(-v_train.T[[j],:] * np.dot(U_train.T[[j],:], x))) for j in range(N))) / N

In [12]:
def objective(x):
    return (((np.linalg.norm(np.dot(A, x) - b)) ** 2) / (2 * N))

#This is the objective function we use for minimization

# Running SGD or GD

### Make sure to run GD before SGD because GD sets random seed, A, and b.

In [ ]:
#Change the random seed here.
np.random.seed(0)

#A = U_train.T #Setting A as pixel data
#b = v_train.T #Setting b as the -1 or 1 array. Reference the 'change labels' cell.

#Adjust stepsize until you get convergence
ss1 = 10 ** -7

#Number of iterations
kmax1 = 100

#Initial xnow point
x1 = np.zeros((784, 1))

#Number of data points -1
ep = 10

#Final1 is final xnow after performing GD, graphData1 is used for plots in next section
final1, graphData1 = gradient_descent(x1, ss1, kmax1, ep)


In [ ]:
#Adjust stepsize until you get convergence
ss2 = 10 ** -7

#Number of iterations
kmax2 = kmax1 * N

#Initial xnow point
x2 = np.zeros((784, 1))

#Number of data points - 1
ep2 = 10

#Final2 is final xnow after performing SGD, graphData2 is used for plots in next section
final2, graphData2 = SGD(x2, ss2, kmax2, ep)


<h1> Graphs <h1>

# Data for GD

In [ ]:
#Make sure you don't get 'nan' values!
print(final1) #Final x array after performing GD

In [ ]:
fig = plt.figure(figsize=(10,7))

#This is the line code
plt.plot(range(0,kmax1+1, math.ceil(kmax1 / ep)), graphData1, color='b',
         marker='X',markersize=10,linestyle='dashed',label="Gradient-Descent",linewidth=3.5)

#General formatting code
plt.legend(loc=1,fontsize=25)
plt.xlabel('Total communication rounds', color='#1C2833',fontsize=30)
plt.ylabel('Log of average loss', color='#1C2833',fontsize=30)
plt.xticks(range(0,kmax1+1, math.ceil(kmax1 / ep)),fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.margins(0.022)

In [ ]:
print(graphData1[-1]) #Checking the value for the last data point. Make sure this is not 0.

# Data for SGD

In [ ]:
#Make sure you don't get 'nan' values!
print(final2)#Final x array after performing SGD

In [ ]:
fig = plt.figure(figsize=(10,7))

plt.plot(range(0,kmax2+1, math.ceil(kmax2 / ep)), graphData2, color='b',
         marker='X',markersize=10,linestyle='dashed',label="SGD",linewidth=3.5)

#General formatting code
plt.legend(loc=1,fontsize=25)
plt.xlabel('Total communication rounds', color='#1C2833',fontsize=30)
plt.ylabel('Log of average loss', color='#1C2833',fontsize=30)
plt.xticks(range(0,kmax2+1, math.ceil(kmax2 / ep)),fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.margins(0.022)

In [ ]:
print(graphData2[-1]) #Checking the value of the last data point. Make sure this is not 0.

# Comparison

In [ ]:
#This graph compares the performance of SGD and Gradient Descent

fig = plt.figure(figsize=(10,7))
#Make sure all kmax inputs are kmax2.

#SGD line
plt.plot(range(0,kmax2+1, math.ceil(kmax2 / ep)), graphData2, color='b',
         marker='X',markersize=10,linestyle='dashed',label="SGD",linewidth=3.5)

#GD line
plt.plot(range(0,kmax2+1, math.ceil(kmax2 / ep)), graphData1, color='g',
         marker='X',markersize=10,linestyle='dashed',label="Gradient-Descent",linewidth=3.5)

plt.legend(loc=1,fontsize=25)
plt.xlabel('Number of Gradient Evaluations', color='#1C2833',fontsize=30)
plt.ylabel('Objective Function Value', color='#1C2833',fontsize=30)
plt.xticks(np.linspace(0, kmax2, 6),fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.margins(0.022)

# Function for Testing

In [ ]:
def my_confu_mat(opt_sol):
    pred_labels = np.zeros((N_test,1))
    for j in range(N_test):
        pred_labels[j][0] = np.sign(np.dot(U_test[:,[j]].T,opt_sol))
    return confusion_matrix(v_test.T, pred_labels)

def precision_score(opt_sol):
    confu_mat = my_confu_mat(opt_sol)
    return (confu_mat[0][0]+confu_mat[1][1])/(confu_mat[1][0]+confu_mat[0][1]+confu_mat[1][1]+confu_mat[0][0])

# Testing our Descent Methods

In [ ]:
#Test Gradient Descent Precision Score
ps_GD = precision_score(final1) 

print("Test score for GD: ", "{0:.2%}".format(ps_GD))

In [ ]:
#Test Stochastic Gradient Descent Precision Score
ps_SGD = precision_score(final2) 

print("Test score for SGD: ", "{0:.2%}".format(ps_SGD))